# CLV Data Mining

In [1]:
# import the required packages
import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

from datetime import datetime

So far the **migration modeling**. Time to get back to our **data mining** methodology. 

We now know what we should do: model CLV during 2004-2006.

We are going to create a **predictive model** with the same independent variables as in the **churn case**.

The difference this time is our **dependent variable**: ``CLV``. 
We will give you the code to create the independent features, since you already learned how to model them during the churn case.

**Two models** will be created:  
- A classification churn model  
- A regression clv model

Because if we would simply regress CLV, our model would be **biased** towards those zeros, leading to a consistent underestimation.

We are going to **predict CLV if customers did not churn**.

(See theory)

# 1. Data Exploration

In [2]:
# import all the datasets
extrel = pd.read_csv("./data/extrel.csv")
extrelty = pd.read_csv("./data/extrelty.csv")
communication = pd.read_csv("./data/communication.csv")
payhistory = pd.read_csv("./data/payhistory.csv")
nameaddr = pd.read_csv("./data/nameaddr.csv")
comclas = pd.read_csv("./data/comclas.csv")
commediu = pd.read_csv("./data/commediu.csv")

In [3]:
# inspect first observation extrel
extrel.head()

,EXTRELNO,EXRELACTCD,EXTRELSTDT,EXRELDATEN
0,26414,CT,2008-01-30,NaT
1,26419,FP,2005-02-26,NaT
2,26424,FP,2005-02-26,2009-12-21
3,26430,FP,2005-02-26,2019-01-28
4,26430,CT,2010-03-04,NaT


In [4]:
# inspect dtypes of columns
extrel.dtypes

EXTRELNO       int64
EXRELACTCD    object
EXTRELSTDT    object
EXRELDATEN    object
dtype: object

In [5]:
# inspect shape
extrel.shape

(26689, 4)

In [6]:
# inspect payhistory
payhistory.head()

,PID,PDATE,PAMT,EXTRELNO,PAYTYPECD,STATUS
0,38,2006-12-28,9.57,19,X,CO
1,39,2006-12-28,41.32,20,X,CO
2,40,2006-12-28,13.10,20,X,CO
3,54091,2006-12-29,2.02,20,X,CO
4,104480,2007-05-16,0.50,20,D,OK


In [7]:
# inspect shape
payhistory.shape

(1205720, 6)

## 2. Time Window

In [4]:
# define the model building time window
end_independent = datetime.strptime("22/11/2016", "%d/%m/%Y")
start_dependent = datetime.strptime("01/01/2017", "%d/%m/%Y")
end_dependent = datetime.strptime("31/12/2019", "%d/%m/%Y")

# 3. Data Preparation

## 3.1. Customers

We need to take a relevant subset of modelable customers just as we did in the churn case. Therefore we have to make **some constraints** in this CLV-case:

1. The donors have to be **active during the independent period**: they needed to start their relationship with the company before the end of the independent period.
2. We are **not interested** in donors that ended their relation before the start of the dependent.
3. Extra condition in comparison to churn case : take only the donors that **made already a transaction by the end of the independent period**.

In [5]:
# convert start date EXTRELSTDT and end date EXRELDATEN to datetimes
extrel["EXTRELSTDT"] = pd.to_datetime(extrel["EXTRELSTDT"], format='%Y-%m-%d')
extrel["EXRELDATEN"] = pd.to_datetime(extrel["EXRELDATEN"], format='%Y-%m-%d')

In [6]:
# extract the active donors
active_customers = extrel[(extrel["EXTRELSTDT"] <= end_independent) & 
                        ((extrel["EXRELDATEN"] > start_dependent) | (extrel["EXRELDATEN"].isnull()))]

In [7]:
# get unique customers
active_customers = active_customers["EXTRELNO"].unique()
# convert to DataFrame 
active_customers = pd.DataFrame(active_customers)
# rename column
active_customers.columns = ["EXTRELNO"]
# sort by EXTRELNO 
active_customers = active_customers.sort_values(by=["EXTRELNO"])

In [8]:
# convert payhistory PDATE to datetime
payhistory["PDATE"] = pd.to_datetime(payhistory["PDATE"], format='%Y-%m-%d')

In [9]:
# extract only those transactions which were made before the end of the independent period
payhistory_ind = payhistory[payhistory["PDATE"] <= end_independent]
# extract only observations with positive transactions
payhistory_ind = payhistory_ind[payhistory_ind["PAMT"] > 0]

In [10]:
# sort data by extrelno and pdate
payhistory_ind = payhistory_ind.sort_values(by=["EXTRELNO", "PDATE"])

In [11]:
# get unique set of customers with transactions during the independent period
transaction_customers = payhistory_ind["EXTRELNO"].unique()
# convert to DataFrame
transaction_customers = pd.DataFrame(transaction_customers)
# rename column
transaction_customers.columns = ["EXTRELNO"]

In [12]:
# merge both tables to get active customers that made transaction during independent period
customers = pd.merge(active_customers, transaction_customers, on="EXTRELNO", how="inner")

In [17]:
# check
customers.head()

,EXTRELNO
0,26414
1,26419
2,26430
3,26431
4,26443


## 3.2. Independent Variables

### 3.2.1. Independent information

For creating the **independent variables** such as *recency*, *frequency*, *monetary value*, etc. we first need to make sure that we will only use information during the **independent period**.

In [13]:
# the payhistory from independent period was already calculated in previous step
payhistory_ind.head()

,PID,PDATE,PAMT,EXTRELNO,PAYTYPECD,STATUS
0,38,2006-12-28,9.57,19,X,CO
1,39,2006-12-28,41.32,20,X,CO
2,40,2006-12-28,13.10,20,X,CO
3,54091,2006-12-29,2.02,20,X,CO
4,104480,2007-05-16,0.50,20,D,OK


In [14]:
# convert communication CONTDATE to datetime
communication["CONTDATE"] = pd.to_datetime(communication["CONTDATE"], format='%Y-%m-%d')

In [15]:
# extract only those transactions which were made before the end of the independent period
communication_ind = communication[communication["CONTDATE"] <= end_independent]

In [16]:
# sort transactions by EXTRELNO and CONTDATE
communication_ind = communication_ind.sort_values(by=["EXTRELNO", "CONTDATE"])

In [17]:
# check
communication_ind.head()

,CONTID,MEDIUMCODE,MNTOPCODE,CLASCODE,EXTRELNO,CONTDIREC,CONTDATE
0,405402,CI,CTS,CT,20,O,2007-05-06
1,430904,CI,CTS,CT,20,O,2007-06-11
2,592732,CI,CTS,CT,20,O,2007-10-18
3,593375,CI,CTS,CT,20,O,2007-10-18
4,618514,CI,CTS,CT,20,O,2007-11-25


### 3.2.2. Frequency

**Frequency**: the number of transactions made by a customer during the independent period

In [18]:
# get the frequency: number of transactions per customer
frequency = payhistory_ind["EXTRELNO"].value_counts().reset_index()
# rename columns
frequency.columns = ["EXTRELNO", "frequency"]

In [24]:
# check
frequency.head()

,EXTRELNO,frequency
0,1773,2131
1,35231,1121
2,34337,642
3,202611,610
4,2980,583


### 3.2.3. Recency

**Recency**: the number of days between a customer's last transaction and the end of the independent period

In [19]:
# get the last transactions of each donor
recency = payhistory_ind.groupby("EXTRELNO").tail(1)[["EXTRELNO", "PDATE"]]
# get the recency in days 
recency["recency"] = recency["PDATE"].apply(lambda x: (end_independent - x).days)
# drop PDATE
recency = recency.drop("PDATE", axis=1)

In [26]:
# check
recency.head()

,EXTRELNO,recency
0,19,3617
120,20,13
275,35,13
397,42,1048
627,43,135


### 3.2.4. Monetary Value

**Monetary value**: the average and total amount of transactions made by a customer during the independent period

In [20]:
# get the total and average monetary value per customer
monetary_value = payhistory_ind.groupby("EXTRELNO").agg({"PAMT": [np.sum, np.mean]}).reset_index()
# rename columns
monetary_value.columns = ["EXTRELNO", "total_value", "average_value"]

In [28]:
# check
monetary_value.head()

,EXTRELNO,total_value,average_value
0,19,9.57,9.570000
1,20,2321.30,19.506723
2,35,2292.40,19.593162
3,42,1563.36,18.611429
4,43,4413.90,19.444493


### 3.2.5. Paytypes

**Paytype**: indication of whether a customer has ever used a particular paytype during the independent period 

         D: order
         E: initiative
         O: sendout
         X: unknown

In [21]:
# get a unique set of paytypes used per donor
paytypes = pd.DataFrame(payhistory_ind.groupby("EXTRELNO").agg({"PAYTYPECD": set}).reset_index())

In [22]:
# create indicators for paytypes used per donor
paytypes["sendout"] = paytypes["PAYTYPECD"].apply(lambda x: 1 if "O" in x else 0)
paytypes["order"] = paytypes["PAYTYPECD"].apply(lambda x: 1 if "D" in x else 0)
paytypes["initiative"] = paytypes["PAYTYPECD"].apply(lambda x: 1 if "E" in x else 0)
paytypes["unknown"] = paytypes["PAYTYPECD"].apply(lambda x: 1 if len(x.difference(set(["O", "D", "E"]))) > 0 else 0)

In [23]:
# drop the paytype column
paytypes = paytypes.drop("PAYTYPECD", axis=1)

In [32]:
# check
paytypes.head()

,EXTRELNO,sendout,order,initiative,unknown
0,19,0,0,0,1
1,20,0,1,0,1
2,35,0,1,0,1
3,42,0,1,0,1
4,43,0,1,0,1


### 3.2.6. Preferred Language

**Preferred mailing language**: get the preferred mailing language of a donor

In [24]:
# get preferred mailing language of donor
language = pd.get_dummies(nameaddr, columns=['LANGUACODE'], drop_first = True)
language = language[['EXTRELNO', 'LANGUACODE_FR', 'LANGUACODE_NL']]
language.head()

,EXTRELNO,LANGUACODE_FR,LANGUACODE_NL
0,19,0,1
1,20,0,1
2,35,0,1
3,42,0,1
4,43,0,1


### 3.2.7. Complaints

Complaint and direction: check if a donor has ever complaint or has given some kind of direction

        CM: Complaint
        I: Direction

In [25]:
# get a unique set om communication types per donor
complaint = communication_ind.groupby("EXTRELNO").agg({"CLASCODE": set, "CONTDIREC": set}).reset_index()
# create indicators for complaint and direction
complaint["complaint"] = complaint["CLASCODE"].apply(lambda x: 1 if "CM" in x else 0)
complaint["direction"] = complaint["CONTDIREC"].apply(lambda x: 1 if "I" in x else 0)
# keep relevant variables
complaint = complaint[["EXTRELNO", "complaint", "direction"]]

In [35]:
# check
complaint.head()

,EXTRELNO,complaint,direction
0,20,0,0
1,35,0,1
2,42,0,1
3,43,0,1
4,60,0,1


We will already store all the independent variables in **one dataset**, which we can than **merge** with the dependent variables.

In [26]:
# create dataset with all independent features
indep_basetable = (pd.merge(pd.DataFrame(customers["EXTRELNO"]), frequency, how="left", on="EXTRELNO")
                    .merge(recency, how="left", on="EXTRELNO")
                    .merge(monetary_value, how="left", on="EXTRELNO")
                    .merge(paytypes, how="left", on="EXTRELNO")
                    .merge(complaint, how="left", on="EXTRELNO")
                    .merge(language, how="left", on="EXTRELNO"))

In [37]:
# check
indep_basetable.head()

,EXTRELNO,frequency,recency,total_value,average_value,sendout,order,initiative,unknown,complaint,direction,LANGUACODE_FR,LANGUACODE_NL
0,26414,117,13,2257.13,19.291709,0,1,0,1,0.0,1.0,0,1
1,26419,47,56,2556.99,54.404043,1,0,0,1,0.0,1.0,0,1
2,26430,119,13,2299.28,19.321681,1,1,0,1,1.0,1.0,0,1
3,26431,14,203,2742.57,195.897857,1,0,0,1,0.0,0.0,0,1
4,26443,116,13,2276.95,19.628879,0,1,0,1,0.0,0.0,0,1


In [38]:
# check dimensions
indep_basetable.shape

(7039, 13)

## 3.3. Dependent Variables

Since we have a **two-phase model** here, we need **two dependent variables**:

- ``CLV`` is in this case defined as the sum of all discounted purchases in the dependent period (3 year)
- ``CHURN`` is in this case defined as a customer who didn't donate anything in the dependent period (CLV=0)

### 3.3.1. CLV

In [31]:
# define the discount rate
DISCOUNT_RATE = 0.05

In [32]:
# get payhistory information from dependent period
payhistory_dep = payhistory[(payhistory["PDATE"] >= start_dependent) & (payhistory["PDATE"] <= end_dependent)]

In [33]:
payhistory_dep

,PID,PDATE,PAMT,EXTRELNO,PAYTYPECD,STATUS
122,3958746,2017-01-10,23.0,20,D,OK
123,3989899,2017-02-08,23.0,20,D,OK
124,4020717,2017-03-08,23.0,20,D,OK
125,4051630,2017-04-10,23.0,20,D,OK
126,4081980,2017-05-08,23.0,20,D,OK
...,...,...,...,...,...,...
1205700,5074768,2019-12-30,30.0,244569,E,OK
1205703,5073823,2019-12-26,25.0,244586,E,OK
1205708,5073817,2019-12-26,25.0,244638,E,OK
1205711,5075134,2019-12-30,25.0,244677,E,OK


**Exercise:**

1. Define a **function** that discounts a certain transaction by making use of the ``DISCOUNT_RATE``. This function should take 2 parameters: the date if the transaction ``PDATE`` and the amount donated ``PAMT``.
2. **Apply** the function on the *payhistory_dep* data set and put the results in a column called ``PAMT_discounted``.
3. Get the **total** discounted amount **per donor** and **check** results.

In [34]:
def get_discounted_transaction(trans_date, trans_val):

    # get discounted transaction
    disc_trans = trans_val / ((1 + DISCOUNT_RATE) ** ((trans_date - start_dependent).days / 365.))

    # return
    return(disc_trans)

payhistory_dep["PAMT_discounted"] = payhistory_dep.apply(lambda x: get_discounted_transaction(x["PDATE"],x["PAMT"]), axis=1)

total_amount = payhistory_dep.groupby("EXTRELNO").sum().reset_index()
total_amount



C:\Users\stefd\AppData\Local\Temp\ipykernel_22364\2966981048.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  payhistory_dep["PAMT_discounted"] = payhistory_dep.apply(lambda x: get_discounted_transaction(x["PDATE"],x["PAMT"]), axis=1)


,EXTRELNO,PID,PAMT,PAMT_discounted
0,20,162553452,828.0,770.965937
1,35,162565113,828.0,770.965937
2,72,54155449,828.0,770.862336
3,81,163543526,2484.0,2303.809274
4,89,166930548,805.0,748.721848
...,...,...,...,...
10906,244569,5074768,30.0,25.922057
10907,244586,5073823,25.0,21.613268
10908,244638,5073817,25.0,21.613268
10909,244677,5075134,25.0,21.601714


In [35]:
clv = total_amount[["EXTRELNO", "PAMT_discounted"]]
clv.columns = ["EXTRELNO", "CLV"]
clv

,EXTRELNO,CLV
0,20,770.965937
1,35,770.965937
2,72,770.862336
3,81,2303.809274
4,89,748.721848
...,...,...
10906,244569,25.922057
10907,244586,21.613268
10908,244638,21.613268
10909,244677,21.601714


### 3.3.2. Churn

**Exercise:**

1. **Merge** clv dataset with customer data and only keep the columns ``EXTRELNO`` and ``CLV``.
2. **Set CLV to zero** for customers that did not make any transactions (i.e., donations) during the dependent period. NOTE: also check for **missing values* and hwo to handle them.
3. Check **churn proportions** and check the difference between mean and standard deviation of **all** donors versus only donors with a **positive CLV**.
4. Plot distribution of CLV of **all** donors versus only donors with a **positive CLV**.
5. Check **how many** observations are left.

In [40]:
# Write your code here
merge = pd.merge(left=customers["EXTRELNO"], right=clv, on="EXTRELNO", how="left")
merge["CLV"] = merge["CLV"].fillna(0)
merge["churned"] = np.where(merge["CLV"] > 0, 0, 1)
merge


,EXTRELNO,CLV,churned
0,26414,770.965937,0
1,26419,771.030576,0
2,26430,484.389230,0
3,26431,777.367765,0
4,26443,770.965937,0
...,...,...,...
7034,232369,770.965937,0
7035,232377,793.760524,0
7036,232395,838.006453,0
7037,232434,770.862336,0


In [47]:
mean_all_donors = merge["CLV"].mean()
sd_all_donors = merge["CLV"].std()

In [48]:
mean_pos_donors = merge.groupby("churned")["CLV"].mean()
mean_pos_donors = mean_pos_donors[0]

sd_pos_donors = merge.groupby("churned")["CLV"].std()
sd_pos_donors = sd_pos_donors[0]

In [49]:
print(f'mean all donors: {mean_all_donors}\nstandard deviation all donors: {sd_all_donors}\nmean positive clv donors: {mean_pos_donors}\nstandard deviation positive clv donors: {sd_pos_donors}')

mean all donors: 690.6549348376129
standard deviation all donors: 361.66105938180493
mean positive clv donors: 754.548606742097
standard deviation positive clv donors: 307.01917009073236


**Remember why we modelled churn in a two-phase model?** 

Because otherwise our sample would be **induced by zero values** for our regression model. However, our sample is induced much stronger by another value: the **default donation contracts**! (i.e., CLV around 770). So should we not tackle this? In fact it might be better to model this seperately, rather than churn: *default_value if P(default), E(CLV) otherwise*. Or even in a **three-phase model**!

But in this case, we will be using the more default way to model CLV, with **churn probability** as secondary input, as this is the most common way. Be aware, however, that you might need to model it slightly different in reality. Always inspect your data!

## 3.4. Basetable

Create the **final basetable** by joining all the independent variables with the dependent variables

In [51]:
# Write your code here
basetable = pd.merge(indep_basetable, merge, on="EXTRELNO", how="left")
basetable.head()

,EXTRELNO,frequency,recency,total_value,average_value,sendout,order,initiative,unknown,complaint,direction,LANGUACODE_FR,LANGUACODE_NL,CLV,churned
0,26414,117,13,2257.13,19.291709,0,1,0,1,0.0,1.0,0,1,770.965937,0
1,26419,47,56,2556.99,54.404043,1,0,0,1,0.0,1.0,0,1,771.030576,0
2,26430,119,13,2299.28,19.321681,1,1,0,1,1.0,1.0,0,1,484.389230,0
3,26431,14,203,2742.57,195.897857,1,0,0,1,0.0,0.0,0,1,777.367765,0
4,26443,116,13,2276.95,19.628879,0,1,0,1,0.0,0.0,0,1,770.965937,0


# 4. Modeling

## 4.1. Split basetable into training and test

First, we are going to **randomly split** our basetable into a training and test set.
        
        training set: this data will be used to train our model
        validation set: check which churn treshold works best
        test set: this data will be used to get an unbiased evaluation of our model
        
For this specific case, we will put 70% of the data into the **training set** and 30% of the data into the **test set**.

In [53]:
# Write your code here
basetable_train, basetable_test = train_test_split(basetable, test_size=0.3, random_state=33, stratify = basetable["churned"])


Now split up into ``X`` and ``Y`` for the **fitting** of your models.

**Remember**: We now have **two models**, so each model needs its own independent variables and dependent variable.
    
For the **churn model** we use ``all`` independent variables.  
For the **clv model** we use ``recency``, ``frequency`` and ``monetary value`` (average + total)

In [56]:
# Write your code here
train_X_churn = basetable_train.drop(["EXTRELNO", "churned", "CLV"], axis=1)
train_y_churn = basetable_train["churned"]

test_X_churn = basetable_test.drop(["EXTRELNO", "churned", "CLV"], axis=1)
test_y_churn = basetable_test["churned"]


In [57]:
train_X_clv = basetable_train[["recency", "frequency", "total_value", "average_value"]]
train_y_clv = basetable_train["CLV"]

test_X_clv = basetable_test[["recency", "frequency", "total_value", "average_value"]]
test_y_clv = basetable_test["CLV"]

## 4.2. Pre-Processing

We will handle the **missing values** and **scaling** the same way we handled them during the churn case: replace missing values (besides recency!) with **0** and create ``indicator`` that you imputed them. Let's first check whether we have missing values in the first place:

In [ ]:
# Write your code here



As you can see there is **no imputation required** for CLV model! How is this possible? We used another **sampling definition**. So we only have to handle missing values in the churn dataset.

In [ ]:
# Write your code here



Let's now perform the **scaling** on BOTH datasets (only features with range outside [0,1], like we did in the churn case):

In [ ]:
# Write your code here



In [ ]:
# Write your code here



## 4.3. Logistic Regression for Churn

Next, we will fit a **logistic regression model** onto the training set. 
This model will try to find a relationship between the independent variables and the dependent variable (``churn``).  

In [ ]:
# Write your code here



Use the fitted model on the test set to get $p(no  Churn)$ for each donor and check results.  

In [ ]:
# Write your code here



We only need the probability of **non-churning**, so select predictions for churn = 0 (i.e. probability of no churn) and **plot** the distribution of the probailities:

In [ ]:
# Write your code here



## 4.4. Linear Regression for CLV

Next, we are going to train a model to **predict CLV**. This model wil use the ``RFM variables`` as its features and will **only** be trained on examples with a **positive CLV**. We only use those positive instances as the bias caused by the zeros is exactly what we're trying to solve: we want to **predict CLV if customers do not churn!**

In [ ]:
# Write your code here



We will now **adjust** those predictions for donors we assume to be churners. As you saw during the churn case, this treshold very important. We now simply use the **standard 50% cut-off**, but you can try out multiple tresholds and select the best one. To do so, you will need a **validation set** as you saw during the acquisition case.

In [ ]:
# Write your code here



# 5. Evaluation

Evaluate by calculating ``MAE`` and ``RMSE`` and comparing against a **baseline model**.

In [ ]:
# Write your code here



In [ ]:
# Write your code here



In [ ]:
# Write your code here



Performance is only slightly better than using baseline model. This is probably because it would have even been better to use a **three-phase model** which models $P(defaultValue)$ seperately. Let's compare the **total CLV** (i.e., all donors) of our migration model to the the total CLV in reality.

In [ ]:
# Write your code here



We are able to predict the **overall CLV** in the next 3 years with a **2.0% error rate**. This means that this approach approximately has **the same performance as migration modeling** on the aggregated level! 

Let's visualize how well our model performs by plotting the true CLV values next to our predicted CLV values. 

In [ ]:
# Write your code here



**A little extra**: you can **order** the customers that have the highest predicted loss $(p(Churn) * E(CLV))$. Doing so, you know what customers will be **the most costly to lose**. These are the customers you need to focus on.

In [ ]:
# Write your code here

